#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter GoogleAds Segmentology Parameters
GoogleAds funnel analysis using Census data.
 1. Wait for <b>BigQuery->->->Census_Join</b> to be created.
 1. Join the <a hre='https://groups.google.com/d/forum/starthinker-assets' target='_blank'>StarThinker Assets Group</a> to access the following assets
 1. Copy <a href='https://datastudio.google.com/c/u/0/reporting/3673497b-f36f-4448-8fb9-3e05ea51842f/' target='_blank'>GoogleAds Segmentology Sample</a>. Leave the Data Source as is, you will change it in the next step.
 1. Click Edit Connection, and change to <b>BigQuery->->->Census_Join</b>.
 1. Or give these instructions to the client.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_read': 'user',  # Credentials used for reading data.
  'customer_id': '',  # Google Ads customer.
  'developer_token': '',  # Google Ads developer token.
  'login_id': '',  # Google Ads login.
  'recipe_project': '',  # Project ID hosting dataset.
  'auth_write': 'service',  # Authorization used for writing data.
  'recipe_slug': '',  # Name of Google BigQuery dataset to create.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute GoogleAds Segmentology
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      'description': 'Create a dataset for bigquery tables.',
      'hour': [
        4
      ],
      'auth': 'user',
      'dataset': {'field': {'name': 'recipe_slug','kind': 'string','description': 'Place where tables will be created in BigQuery.'}}
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'function': 'Pearson Significance Test',
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}}
      }
    }
  },
  {
    'google_api': {
      'auth': 'user',
      'api': 'googleads',
      'version': 'v5',
      'function': 'customers.googleAds.search',
      'kwargs': {
        'customerId': {'field': {'name': 'customer_id','kind': 'string','description': 'Google Ads customer.','default': ''}},
        'body': {
          'query': 'SELECT           campaign.name,           ad_group.name,           segments.geo_target_postal_code,           metrics.impressions,           metrics.clicks,           metrics.conversions,           metrics.interactions           FROM user_location_view         '
        }
      },
      'headers': {
        'developer-token': {'field': {'name': 'developer_token','kind': 'string','description': 'Google Ads developer token.','default': ''}},
        'login-customer-id': {'field': {'name': 'login_id','kind': 'string','description': 'Google Ads login.','default': ''}}
      },
      'iterate': True,
      'results': {
        'bigquery': {
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
          'table': 'GoogleAds_KPI',
          'schema': [
            {
              'name': 'userLocationView',
              'type': 'RECORD',
              'mode': 'NULLABLE',
              'fields': [
                {
                  'name': 'resourceName',
                  'type': 'STRING',
                  'mode': 'NULLABLE'
                }
              ]
            },
            {
              'name': 'segments',
              'type': 'RECORD',
              'mode': 'NULLABLE',
              'fields': [
                {
                  'name': 'geoTargetPostalCode',
                  'type': 'STRING',
                  'mode': 'NULLABLE'
                }
              ]
            },
            {
              'name': 'metrics',
              'type': 'RECORD',
              'mode': 'NULLABLE',
              'fields': [
                {
                  'name': 'interactions',
                  'type': 'INTEGER',
                  'mode': 'NULLABLE'
                },
                {
                  'name': 'impressions',
                  'type': 'INTEGER',
                  'mode': 'NULLABLE'
                },
                {
                  'name': 'conversions',
                  'type': 'INTEGER',
                  'mode': 'NULLABLE'
                },
                {
                  'name': 'clicks',
                  'type': 'INTEGER',
                  'mode': 'NULLABLE'
                }
              ]
            },
            {
              'name': 'adGroup',
              'type': 'RECORD',
              'mode': 'NULLABLE',
              'fields': [
                {
                  'name': 'name',
                  'type': 'STRING',
                  'mode': 'NULLABLE'
                },
                {
                  'name': 'resourceName',
                  'type': 'STRING',
                  'mode': 'NULLABLE'
                }
              ]
            },
            {
              'name': 'campaign',
              'type': 'RECORD',
              'mode': 'NULLABLE',
              'fields': [
                {
                  'name': 'name',
                  'type': 'STRING',
                  'mode': 'NULLABLE'
                },
                {
                  'name': 'resourceName',
                  'type': 'STRING',
                  'mode': 'NULLABLE'
                }
              ]
            }
          ]
        }
      }
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'from': {
        'query': 'SELECT            campaign.name AS Campaign,            adGRoup.name AS Ad_Group,            segments.geoTargetPostalCode AS Postal_Code,            SAFE_DIVIDE(metrics.impressions, SUM(metrics.impressions) OVER()) AS Impression_Percent,            SAFE_DIVIDE(metrics.clicks, metrics.impressions) AS Click_Percent,            SAFE_DIVIDE(metrics.conversions, metrics.impressions) AS Conversion_Percent,            SAFE_DIVIDE(metrics.interactions, metrics.impressions) AS Interaction_Percent,            metrics.impressions AS Impressions          FROM            `{project}.{dataset}.GoogleAds_KPI`;        ',
        'parameters': {
          'project': {'field': {'name': 'recipe_project','kind': 'string','description': 'Project ID hosting dataset.'}},
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','description': 'Place where tables will be created in BigQuery.'}}
        },
        'legacy': False
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','description': 'Place where tables will be written in BigQuery.'}},
        'view': 'GoogleAds_KPI_Normalized'
      }
    }
  },
  {
    'census': {
      'auth': 'user',
      'normalize': {
        'census_geography': 'zip_codes',
        'census_year': '2018',
        'census_span': '5yr'
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'type': 'view'
      }
    }
  },
  {
    'census': {
      'auth': 'user',
      'correlate': {
        'join': 'Postal_Code',
        'pass': [
          'Campaign',
          'Ad_Group'
        ],
        'sum': [
          'Impressions'
        ],
        'correlate': [
          'Impression_Percent',
          'Click_Percent',
          'Conversion_Percent',
          'Interaction_Percent'
        ],
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'table': 'GoogleAds_KPI_Normalized',
        'significance': 80
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'type': 'view'
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
